# Tutorial 9 - LLM Tracing with OpenTelemetry and Arize Phoenix

## Where You Are in the Learning Journey

```
 Tutorials 1-5      Tutorials 6-8      Tutorial 9
 RAG Fundamentals   Agent Extension    LLM Tracing
 (the retrieval     (ReAct, Reflection (you are here)
  pipeline)          State Mgmt)
```

**What this tutorial adds:** observability for the RAG pipeline built in
Tutorials 1-8.  You will learn how to record every retrieval call, every LLM
call, and every end-to-end request as structured, searchable trace data.

**What you will learn in this tutorial:**
- What observability is and why LLM systems need it
- The three OpenTelemetry building blocks: Spans, Traces, Exporters
- How to create spans manually to record any operation
- How to attach metadata (attributes) and error information to spans
- How to nest spans to model parent-child relationships (e.g., one RAG request
  contains one retrieval span and one generation span)
- How to send traces to Arize Phoenix for visualization
- How to auto-instrument OpenAI calls with one line using OpenInference

**Prerequisites:** Tutorial 1 (dense RAG baseline).  Tutorials 2-8 are
helpful context but not required.

```mermaid
flowchart TD
    Q[User Question] --> P[RAG Pipeline span]
    P --> R[retrieval span]
    P --> G[generation span]
    R --> E1[Exporter]
    G --> E1
    P --> E1
    E1 --> PH[Arize Phoenix UI]
```


## What Is Observability and Why Do LLM Systems Need It?

### The Problem with Black-Box LLM Systems

When a RAG pipeline gives a wrong answer, you need to know **where the
failure happened**:

- Did the retriever return irrelevant chunks?
- Did the LLM ignore the context?
- Did the embedding model fail on an unusual query?
- Was a step unexpectedly slow?

Without instrumentation, the only thing you see is the final answer.
With instrumentation, you see a complete record of every step.

### What Is Observability?

**Observability** is the practice of making the internal state of a system
visible from the outside.  The three traditional pillars are:

| Pillar | What it captures | Example |
|--------|------------------|---------|
| Logs   | Discrete events with a timestamp | `ERROR: retriever returned 0 results` |
| Metrics | Numeric measurements over time | `retrieval_latency_ms = 230` |
| Traces | Causally connected spans for one request | `RAG request -> retrieval -> generation` |

This tutorial focuses on **traces** because they are the most useful tool
for understanding *why* an individual request went wrong.

### What Is OpenTelemetry?

**OpenTelemetry (OTel)** is an open-source standard for producing and
collecting observability data.  It is vendor-neutral: you write your
instrumentation once and send the data to any backend (Arize Phoenix,
Jaeger, Honeycomb, Datadog, etc.).

The Python SDK (`opentelemetry-api` + `opentelemetry-sdk`) provides:
- A `Tracer` object that creates spans
- A `TracerProvider` that configures how spans are processed and exported
- Exporters that forward finished spans to a backend over the network


## Core OpenTelemetry Concepts: Spans, Traces, and Exporters

### Concept 1 - Span

A **span** is the basic unit of a trace.  It represents **one named,
timed operation**.

Every span has:
- A **name** that identifies the operation (e.g., `"retrieval"`)
- A **start time** and **end time** (latency = end - start)
- **Attributes** - key-value pairs attached as metadata
- A **status** - either OK or ERROR
- Optional **events** - timestamped notes added during execution

```
Span: retrieval
  start_time   : 2024-01-15 10:00:00.000
  end_time     : 2024-01-15 10:00:00.230
  duration     : 230 ms
  attributes:
    input.value          -> "remote work VPN policy"
    retrieval.documents  -> 5
  status       : OK
```

### Concept 2 - Trace

A **trace** is a collection of spans connected by a shared **trace ID**.
Spans within a trace can be nested: a **parent span** represents a
high-level operation, and **child spans** represent sub-operations inside it.

```
Trace ID: abc123
  rag-pipeline (parent)          0 ms - 450 ms
    retrieval (child)            5 ms - 235 ms
    generation (child)         235 ms - 450 ms
```

The nesting is automatic: when you create a child span while a parent span
is active in the current thread, OTel links them via the parent's span ID.

### Concept 3 - Exporter

An **exporter** receives finished spans and forwards them to a backend.
Two exporters are used in this tutorial:

| Exporter | When to use | Where spans go |
|----------|-------------|----------------|
| `ConsoleSpanExporter` | Development, learning | Printed to stdout |
| `OTLPSpanExporter` | Production, visualization | Arize Phoenix (or any OTLP backend) |

### Concept 4 - TracerProvider

The **TracerProvider** is the entry point for all tracing.  It:
- Creates `Tracer` objects (one per component/library)
- Holds the list of `SpanProcessor` objects (which call the exporter)
- Attaches `Resource` metadata (e.g., service name) to every span

You configure the provider once at startup with `configure_tracing()`
from the `rag_tutorials.tracing` module, then create tracers as needed.


In [ ]:
import importlib
import os
from pathlib import Path
import shutil
import subprocess
import sys

from dotenv import load_dotenv

if shutil.which("uv") is None:
    print("uv not found. Installing with pip...")
    subprocess.run([sys.executable, "-m", "pip", "install", "uv"], check=True)

cwd = Path.cwd().resolve()
repo_root = next(
    (path for path in [cwd, *cwd.parents] if (path / "pyproject.toml").exists() and (path / "src").exists()),
    cwd,
)
os.chdir(repo_root)
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

REQUIRED_PACKAGES = [
    "openai", "chromadb", "numpy", "pandas", "rank_bm25",
    "sentence_transformers", "dotenv",
    "opentelemetry", "opentelemetry.sdk",
    "openinference.instrumentation.openai",
]
PIP_NAME_MAP = {
    "rank_bm25": "rank-bm25",
    "sentence_transformers": "sentence-transformers",
    "dotenv": "python-dotenv",
    "opentelemetry": "opentelemetry-api",
    "opentelemetry.sdk": "opentelemetry-sdk",
    "openinference.instrumentation.openai": "openinference-instrumentation-openai",
}

def find_missing(packages):
    importlib.invalidate_caches()
    return [pkg for pkg in packages if importlib.util.find_spec(pkg) is None]

missing = find_missing(REQUIRED_PACKAGES)
if missing:
    print("Missing packages:", missing)
    subprocess.run(["uv", "sync", "--extra", "tracing"], check=True)

missing_after_sync = find_missing(REQUIRED_PACKAGES)
if missing_after_sync:
    pip_targets = [PIP_NAME_MAP.get(pkg, pkg) for pkg in missing_after_sync]
    subprocess.run([sys.executable, "-m", "pip", "install", *pip_targets], check=True)

final_missing = find_missing(["openai", "chromadb", "numpy", "pandas"])
if final_missing:
    raise ImportError(f"Dependencies still missing: {final_missing}")

from rag_tutorials.io_utils import load_handbook_documents, load_queries
from rag_tutorials.chunking import semantic_chunk_documents
from rag_tutorials.pipeline import build_dense_retriever
from rag_tutorials.qa import answer_with_context
from rag_tutorials.tracing import (
    configure_tracing,
    get_tracer,
    traced_retrieval,
    traced_generation,
    build_traced_rag_pipeline,
    ATTR_INPUT_VALUE,
    ATTR_OUTPUT_VALUE,
    ATTR_LLM_MODEL_NAME,
    ATTR_RETRIEVAL_DOCUMENTS,
)

load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise EnvironmentError("OPENAI_API_KEY is required")

embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
chat_model = os.getenv("OPENAI_CHAT_MODEL", "gpt-4.1-mini")

handbook_path = Path("data/handbook_manual.txt")
queries_path = Path("data/queries.jsonl")
if not handbook_path.exists() or not queries_path.exists():
    raise FileNotFoundError("Run: uv run python scripts/generate_data.py")

documents = load_handbook_documents(handbook_path)
queries = load_queries(queries_path)
chunks = semantic_chunk_documents(documents)
dense_retriever, _ = build_dense_retriever(
    chunks=chunks,
    collection_name="tracing_tutorial_dense",
    embedding_model=embedding_model,
)

print("Setup complete.")
print(f"  Documents  : {len(documents)}")
print(f"  Chunks     : {len(chunks)}")
print(f"  Queries    : {len(queries)}")
print(f"  Chat model : {chat_model}")


## Part 1 - Manual Span Creation (No Backend Required)

Before connecting to Phoenix, learn how spans work by creating them
manually and reading the output printed to stdout.

The three steps for any span:

1. Call `configure_tracing()` to set up the provider (once per session)
2. Call `get_tracer(name)` to get a tracer for your component
3. Use `tracer.start_as_current_span(name)` as a context manager around
   the code you want to measure

Anything inside the `with` block is timed automatically.  Attributes are
key-value pairs you attach to describe what the span measured.


In [ ]:
# Step 1 - Set up the provider.
# With no arguments, spans are printed to stdout (ConsoleSpanExporter).
# This is useful for learning and development.

configure_tracing(service_name="rag-tracing-tutorial")

# Step 2 - Get a tracer.  The name is a label for this component.
tracer = get_tracer("tutorial.manual")

# Step 3 - Create a span.
# 'with tracer.start_as_current_span(name) as span:' starts the timer.
# The span ends automatically when the 'with' block exits.

with tracer.start_as_current_span("first-span") as span:
    # Attach metadata with set_attribute(key, value).
    # Keys are plain strings; values can be strings, ints, or floats.
    span.set_attribute("example.greeting", "hello from the span")
    span.set_attribute("example.number", 42)

    # Your normal code goes here.
    result = 6 * 7
    span.set_attribute("example.result", result)

# After the 'with' block, the span is finished and exported.
# Look for 'first-span' in the JSON output above.
print(f"Span finished. Computed result: {result}")


### Reading the Console Exporter Output

The JSON block printed above contains the complete span.  Key fields to
look for:

| JSON field | What it means |
|------------|---------------|
| `name` | The span name you passed to `start_as_current_span` |
| `start_time` | When the span started (nanosecond timestamp) |
| `end_time` | When the span ended |
| `attributes` | Key-value pairs attached with `set_attribute` |
| `status` | `StatusCode.OK` when no error was recorded |
| `context.trace_id` | Unique ID shared by all spans in one trace |
| `context.span_id` | Unique ID for this individual span |
| `parent_id` | Span ID of the parent (None for a root span) |
| `resource` | Service-level metadata (service name, etc.) |

The `trace_id` links spans that belong to the same request.  When you
create a child span inside a parent span, the child automatically gets the
same `trace_id` and records the parent's `span_id` as its `parent_id`.


### Parent-Child Span Relationships

Nesting is the key feature that makes traces useful.  A parent span
represents the whole operation; child spans represent sub-steps inside it.

The rule is simple: **any span started while another span is active becomes
its child automatically.**  There is no explicit linking call required.

The cell below creates a `rag-request` parent with `retrieval` and
`generation` children, mirroring what the RAG pipeline actually does.


In [ ]:
# Parent-child span demo
# Create a root span for the whole request, then child spans for each step.

with tracer.start_as_current_span("rag-request") as root_span:
    root_span.set_attribute(ATTR_INPUT_VALUE, "What is the VPN policy?")

    # The retrieval child span is nested inside rag-request.
    with tracer.start_as_current_span("retrieval") as retrieval_span:
        retrieval_span.set_attribute(ATTR_INPUT_VALUE, "VPN policy")
        # Simulating work
        retrieval_span.set_attribute(ATTR_RETRIEVAL_DOCUMENTS, 5)

    # The generation child span is also nested inside rag-request.
    with tracer.start_as_current_span("generation") as generation_span:
        generation_span.set_attribute(ATTR_LLM_MODEL_NAME, chat_model)
        generation_span.set_attribute(ATTR_INPUT_VALUE, "What is the VPN policy?")
        # Simulating work
        generation_span.set_attribute(ATTR_OUTPUT_VALUE, "VPN is required for all remote access.")

print("Trace complete.")
print("Look for rag-request, retrieval, and generation spans above.")
print("Notice that retrieval and generation share the same trace_id as rag-request.")


### Span Events and Error Recording

A **span event** is a timestamped note added inside an active span.  Use
events to mark significant moments that are not operations in their own
right (for example, "cache miss" or "context truncated").

When an exception occurs inside a span, you can record it with
`span.record_exception(exc)` and set the status to ERROR.  This lets the
observability backend filter to failed requests.


In [ ]:
from opentelemetry.trace import StatusCode

# Demo 1: span events
with tracer.start_as_current_span("retrieval-with-events") as span:
    span.set_attribute(ATTR_INPUT_VALUE, "international work limit")

    # add_event records a note at the current timestamp
    span.add_event("cache_miss", {"cache_key": "international work limit"})

    # Simulate fetching from the retriever
    span.add_event("retrieved", {"count": 3})
    span.set_attribute(ATTR_RETRIEVAL_DOCUMENTS, 3)

print("Span with events written.  Look for 'events' in the output.")

# Demo 2: error recording
import contextlib

with tracer.start_as_current_span("retrieval-with-error") as span:
    span.set_attribute(ATTR_INPUT_VALUE, "leave policy")
    with contextlib.suppress(ValueError):
        try:
            raise ValueError("Chroma connection failed")
        except ValueError as exc:
            span.record_exception(exc)
            span.set_status(StatusCode.ERROR, str(exc))
            raise

print("Error span written.  Look for 'status': 'StatusCode.ERROR' in the output.")


## Part 2 - Arize Phoenix Setup

### What Is Arize Phoenix?

**Arize Phoenix** is an open-source LLM observability platform.  It
provides:
- A local web server that collects traces sent via the OTLP protocol
- A UI to search, filter, and inspect every span
- Built-in support for LLM-specific attributes (prompt, response, model)
- The ability to replay and compare requests side by side

Phoenix speaks the same OTLP protocol as the OTel SDK, so you can switch
from the console exporter to Phoenix by changing one argument in
`configure_tracing`.

### Starting Phoenix Locally

Run this command in a terminal (not in the notebook) to start Phoenix:

```bash
uv run python -m phoenix.server.main
```

Phoenix starts on port 6006 by default.  Open `http://localhost:6006` in
your browser to see the UI.  Leave the terminal running while you use the
notebook.

You can also start Phoenix programmatically from inside the notebook using
the `arize-phoenix-otel` package.  The cell below shows both approaches.


In [ ]:
# To send traces to Phoenix, change the configure_tracing call to include
# the Phoenix OTLP endpoint:
#
#   configure_tracing(
#       endpoint="http://localhost:6006/v1/traces",
#       service_name="rag-tracing-tutorial",
#   )
#
# The rest of the notebook code is identical regardless of which backend
# you use.  The exporter is the only thing that changes.
#
# Alternatively, use arize-phoenix-otel for a one-line setup that also
# starts a local Phoenix server and registers the exporter:
#
#   import phoenix as px
#   from phoenix.otel import register
#
#   px.launch_app()   # starts Phoenix at http://localhost:6006
#   register()        # configures the global OTel provider for Phoenix
#
# For this notebook the ConsoleSpanExporter (already configured above)
# is used so you can run everything without a running Phoenix instance.

print("Phoenix setup notes printed.")
print("To enable Phoenix: uncomment the configure_tracing call with the endpoint.")


## Part 3 - Tracing the RAG Pipeline with the Shared Module

The `rag_tutorials.tracing` module provides three ready-to-use wrappers
that you can drop in around any existing retriever or answer function:

| Function | What it wraps | Span name created |
|----------|--------------|-------------------|
| `traced_retrieval(retriever, tracer)` | Any retriever callable | `retrieval` |
| `traced_generation(answer_fn, tracer)` | Any generation callable | `generation` |
| `build_traced_rag_pipeline(retriever, answer_fn, tracer)` | Full pipeline | `rag-pipeline` (parent) + two children |

Each wrapper function takes your existing callable and returns a new callable
with identical input/output behaviour.  The only difference is that every
call now creates an OTel span.

### Novice Trace: Watching a Real Retrieval Span

The cell below wraps the dense retriever from Tutorial 1 with
`traced_retrieval` and runs one query.  Watch the console output to see
the span that is created.


In [ ]:
# Wrap the dense retriever with a tracing span.
# The wrapper has the same interface as dense_retriever:
#   (question: str, top_k: int = ...) -> list[RetrievalResult]

retrieval_tracer = get_tracer("tutorial.retrieval")
traced_dense = traced_retrieval(dense_retriever, retrieval_tracer)

question = "What is the maximum number of days an employee can work internationally?"

print("Running retrieval with tracing...")
print(f"Question: {question}")
print()

results = traced_dense(question, top_k=3)

print(f"Retrieved {len(results)} chunks:")
for i, r in enumerate(results, start=1):
    print(f"  Chunk {i} [{r.chunk_id}] (score={r.score:.3f}): {r.text[:80]}...")

print()
print("Look in the console output above this cell for the 'retrieval' span JSON.")
print(f"The span should show input.value='{question[:50]}...' and retrieval.documents=3.")


### Running the Full Traced Pipeline

Now build the end-to-end traced pipeline using `build_traced_rag_pipeline`.
This creates three spans for every question:
1. A parent `rag-pipeline` span that contains the whole request
2. A child `retrieval` span (inside the parent)
3. A child `generation` span (inside the parent)

The parent and child spans all share the same `trace_id`, so Phoenix (and
any other OTLP backend) can display them as a single flame-graph view.


In [ ]:
# Build a fully traced RAG pipeline from the shared modules.

pipeline_tracer = get_tracer("tutorial.pipeline")

pipeline = build_traced_rag_pipeline(
    retriever=dense_retriever,
    answer_fn=answer_with_context,
    tracer=pipeline_tracer,
    model_name=chat_model,
)

question = "What must an employee do before working from another country for more than 14 days?"

print("Running full traced pipeline...")
print(f"Question: {question}")
print()

answer = pipeline(question)

print("ANSWER:", answer)
print()
print("The console output above contains three spans:")
print("  1. 'retrieval'   - child span for the retrieval step")
print("  2. 'generation'  - child span for the LLM call")
print("  3. 'rag-pipeline'- parent span wrapping both children")
print("All three spans share the same trace_id.")


## Part 4 - Auto-Instrumentation with OpenInference

### What Is OpenInference?

**OpenInference** is a set of OTel instrumentation libraries for popular
LLM frameworks.  Instead of manually adding spans everywhere, you call one
setup function and every LLM call in your process is traced automatically.

The `openinference-instrumentation-openai` package instruments the OpenAI
Python SDK.  After calling `OpenAIInstrumentor().instrument()`, every
`client.chat.completions.create(...)` call produces a span that records:

| Attribute | What it captures |
|-----------|------------------|
| `llm.model_name` | The model used (e.g. `gpt-4.1-mini`) |
| `input.value` | The full prompt sent to the model |
| `output.value` | The full response from the model |
| `llm.token_count.prompt` | Number of prompt tokens |
| `llm.token_count.completion` | Number of completion tokens |

This means you get detailed LLM call tracing without modifying any of the
code that calls the OpenAI client.

### Important: Call Order

You must call `configure_tracing()` (or Phoenix's `register()`) **before**
calling `OpenAIInstrumentor().instrument()`.  The instrumentor patches the
OpenAI SDK so that every new call reports to the currently active
TracerProvider.  If the provider is not set yet, spans are silently dropped.

```
Correct order:
  1. configure_tracing(endpoint=...)   or   register()
  2. OpenAIInstrumentor().instrument()
  3. Your application code
```


In [ ]:
# Enable automatic OpenAI instrumentation.
# After this call, every client.chat.completions.create() produces a span
# with the prompt, response, model name, and token counts included.

from openinference.instrumentation.openai import OpenAIInstrumentor

# configure_tracing was already called in the setup cell above,
# so the provider is already active.  Just instrument OpenAI now.
OpenAIInstrumentor().instrument()

print("OpenAI auto-instrumentation is active.")
print("Every OpenAI API call from this point on will produce a span.")


In [ ]:
# Run one question through the pipeline.
# Because OpenAIInstrumentor is active, the internal OpenAI call inside
# answer_with_context will produce an additional automatic span with the
# full prompt and response captured.

auto_tracer = get_tracer("tutorial.auto")

auto_pipeline = build_traced_rag_pipeline(
    retriever=dense_retriever,
    answer_fn=answer_with_context,
    tracer=auto_tracer,
    model_name=chat_model,
)

question = "What is the remote work VPN requirement?"

print(f"Question: {question}")
print()

answer = auto_pipeline(question)

print("ANSWER:", answer)
print()
print("Console output now shows an additional span from OpenAI auto-instrumentation.")
print("That span captures the full prompt text and token usage automatically.")


## Comparing Console Output to a Phoenix UI View

When you switch `configure_tracing` to use the Phoenix endpoint:

```python
configure_tracing(
    endpoint="http://localhost:6006/v1/traces",
    service_name="rag-tracing-tutorial",
)
```

Every span in the console output above would instead appear in the Phoenix
UI at `http://localhost:6006`.  The UI shows:

- A **Traces** table listing every request with its latency and status
- A **Trace Detail** view with a flame graph showing the parent and child
  spans side by side
- **Attribute inspection**: click any span to see all its attributes
- **Token usage charts**: total prompt and completion tokens over time
- **Error filtering**: filter to only show traces with ERROR status

The data you see in Phoenix is exactly the same as what the console exporter
prints - only the presentation is different.

### Switching Between Backends

The pattern for switching from console to Phoenix is:

```
Development  -> configure_tracing()                         (stdout)
Local UI     -> configure_tracing(endpoint="http://localhost:6006/v1/traces")
Cloud        -> configure_tracing(endpoint="https://<your-phoenix-cloud>/v1/traces")
```

Everything else (tracer creation, span wrappers, instrumentors) stays the same.


## Learning Checkpoint: LLM Tracing

### What Works

- The `configure_tracing` / `get_tracer` / `start_as_current_span` pattern
  gives you precise timing and metadata for any operation with very little
  code added.
- Parent-child span nesting is automatic: creating a span inside an active
  span links them by trace ID without any explicit join call.
- The `traced_retrieval` and `traced_generation` wrappers in
  `rag_tutorials.tracing` instrument the full RAG pipeline with two function
  calls.
- `OpenAIInstrumentor().instrument()` captures the complete prompt and
  response for every OpenAI API call without touching the call sites.
- Switching from the console exporter to Phoenix requires changing one
  argument in `configure_tracing`; all other code stays identical.

### What Does Not Work Out of the Box

- The console exporter is good for learning but hard to query at scale.
  Phoenix (or another OTLP backend) is needed for production use.
- The wrappers in this tutorial record span-level metadata but not
  chunk-level embedding vectors or full retrieved text.  Storing those
  requires additional `set_attribute` calls inside `traced_retrieval`.
- OTel context propagation across async tasks or multiple processes
  requires additional setup (context propagators, HTTP header injection).

### What You Have Learned

- Span: the unit of a trace; one named, timed, attributed operation
- Trace: a tree of spans sharing a trace ID; represents one request
- TracerProvider: configures how spans are created and exported
- Exporter: receives finished spans and sends them to a backend
- Auto-instrumentation: patches a library at runtime so every call is traced
- Arize Phoenix: open-source OTLP backend with a UI for LLM traces
